In [ ]:

!pip install googletrans==4.0.0-rc1 \
            diffusers \
            huggingface_hub \
            torch \
            flask \
            Pillow \
            flask-ngrok


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━

In [ ]:

!pip install flask pyngrok googletrans==4.0.0-rc1 diffusers transformers accelerate huggingface_hub
!ngrok config add-authtoken <Your_NGORK_Token>


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:

from googletrans import Translator
from diffusers import StableDiffusionPipeline
from huggingface_hub import login
from pyngrok import ngrok
from flask import Flask, request, jsonify, render_template_string
import torch
import io
import base64
from PIL import Image
import threading

# Initialize the Flask app
app = Flask(__name__)

# Function to translate the input text to a desired language
def get_translation(text, dest_lang):
    translator = Translator()
    translated_text = translator.translate(text, dest=dest_lang)
    return translated_text.text

# Configuration class with parameters
class CFG:
    device = "cuda" if torch.cuda.is_available() else "cpu"           # Use GPU if available
    seed = 42
    image_gen_steps = 35                                              # Number of steps for image generation
    image_gen_model_id = "stable-diffusion-v1-5/stable-diffusion-v1-5"  # Model ID from Hugging Face
    image_gen_size = (600, 600)                                         # Desired output image size
    image_gen_guidance_scale = 9                                        # Guidance scale for controlling image variation
    api_key = "Your_Hugging_Face_token"                   # Replace with your Hugging Face API key
    port = 5000                                                       # Port for running Flask

# Login to Hugging Face Hub using your API key
login(CFG.api_key)

# Load the Stable Diffusion model from Hugging Face Hub
pipe = StableDiffusionPipeline.from_pretrained(
    CFG.image_gen_model_id,
    torch_dtype=torch.float16,               # Use half-precision if using GPU
    token=CFG.api_key                        # Use 'token' instead of deprecated 'use_auth_token'
)

# Move the model to the appropriate device (GPU or CPU)
pipe = pipe.to(CFG.device)

# Function to generate multiple images using the model
def generate_images(prompt, model, num_images=3):
    images = []
    for _ in range(num_images):
        # Generate one image using the model with specified parameters
        image = model(
            prompt,
            num_inference_steps=CFG.image_gen_steps,
            guidance_scale=CFG.image_gen_guidance_scale
        ).images[0]
        # Resize the image if needed to the configured size
        image = image.resize(CFG.image_gen_size)
        images.append(image)
    return images

# HTML structure for the web interface, including CSS and JavaScript
html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Multilingual Image Generation</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f7f8fc;
            margin: 0;
            padding: 20px;
            color: #333;
        }
        .container {
            max-width: 800px;
            margin: 0 auto;
            background-color: #fff;
            padding: 20px;
            border-radius: 15px;
            box-shadow: 0 8px 16px rgba(0, 0, 0, 0.2);
            transition: transform 0.2s ease-in-out;
        }
        .container:hover {
            transform: scale(1.02);
        }
        h1 {
            text-align: center;
            font-size: 28px;
            color: #4CAF50;
        }
        label {
            display: block;
            margin-top: 20px;
            font-size: 18px;
            color: #555;
        }
        input[type="text"], select {
            width: 100%;
            padding: 12px;
            margin-top: 10px;
            margin-bottom: 20px;
            border: 2px solid #ccc;
            border-radius: 8px;
            font-size: 16px;
        }
        button {
            width: 100%;
            padding: 12px;
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 8px;
            font-size: 18px;
            cursor: pointer;
            transition: background-color 0.3s ease-in-out;
        }
        button:hover {
            background-color: #45a049;
        }
        .output-images {
            display: flex;
            justify-content: space-between;
            gap: 30px;
            margin-top: 30px;
        }
        .output-images div {
            display: flex;
            flex-direction: column;
            align-items: center;
            width: 30%;
            transition: transform 0.2s ease;
        }
        .output-images div:hover {
            transform: scale(1.05);
        }
        .output-images img {
            max-width: 100%;
            border-radius: 15px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            margin-bottom: 12px;
        }
        .download-btn {
            display: block;
            margin-top: 10px;
            padding: 10px;
            background-color: #007bff;
            color: white;
            text-align: center;
            border-radius: 8px;
            text-decoration: none;
            font-size: 16px;
            transition: background-color 0.3s ease-in-out;
        }
        .download-btn:hover {
            background-color: #0056b3;
        }
        .loading {
            display: none;
            text-align: center;
            margin-top: 20px;
        }
        .loading .spinner {
            border: 8px solid #f3f3f3;
            border-top: 8px solid #4CAF50;
            border-radius: 50%;
            width: 50px;
            height: 50px;
            animation: spin 1s linear infinite;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
    </style>
</head>
<body>
<div class="container">
    <h1>Multilingual Image Generation</h1>
    <label for="textPrompt">Enter Text Prompt:</label>
    <input type="text" id="textPrompt" placeholder="Type your prompt here..." required>
    <label for="languageSelect">Select Language:</label>
    <select id="languageSelect">
        <option value="en">English</option>
    </select>
    <button onclick="generateImages()">Generate Images</button>
    <div class="loading" id="loading">Generating images, please wait...</div>
    <div class="output-images" id="outputImages">
        <!-- Generated images will appear here -->
    </div>
</div>
<script>
    function generateImages() {
        const prompt = document.getElementById('textPrompt').value;
        const language = document.getElementById('languageSelect').value;
        if (prompt === "") {
            alert("Please enter a prompt.");
            return;
        }
        document.getElementById('loading').style.display = 'block';
        document.getElementById('outputImages').innerHTML = '';
        fetch('/generate-images', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ prompt: prompt, language: language }),
        })
        .then(response => response.json())
        .then(data => {
            document.getElementById('loading').style.display = 'none';
            data.images.forEach((image, index) => {
                const imgElement = `<div>
                    <img src="data:image/png;base64,${image}" alt="Generated Image ${index+1}">
                    <a href="data:image/png;base64,${image}" download="generated_image_${index+1}.png" class="download-btn">Download Image ${index+1}</a>
                </div>`;
                document.getElementById('outputImages').innerHTML += imgElement;
            });
        })
        .catch(error => {
            document.getElementById('loading').style.display = 'none';
            console.error('Error:', error);
        });
    }
</script>
</body>
</html>
"""

# Flask route to serve the HTML interface
@app.route('/')
def index():
    return render_template_string(html_code)

# Flask route to handle image generation requests
@app.route('/generate-images', methods=['POST'])
def generate_images_endpoint():
    data = request.json
    prompt = data['prompt']
    language = data['language']

    # Translate the input prompt to English
    translated_prompt = get_translation(prompt, "en")

    # Generate images using the translated prompt
    images = generate_images(translated_prompt, pipe, num_images=3)

    # Convert each generated image to a base64-encoded string
    image_data = []
    for image in images:
        img_io = io.BytesIO()
        image.save(img_io, 'PNG')
        img_io.seek(0)
        img_base64 = base64.b64encode(img_io.read()).decode('utf-8')
        image_data.append(img_base64)

    # Return the images in JSON format
    return jsonify({'images': image_data})

# Function to start the Flask server in a separate thread
def start_flask():
    # Run with host set to "0.0.0.0" so it is externally accessible and use keyword argument for port
    app.run(host='0.0.0.0', port=CFG.port)

# Start Flask in a background thread
threading.Thread(target=start_flask).start()

# Start an Ngrok tunnel pointing to the configured port
public_url = ngrok.connect(CFG.port)
print(f"\nApp is running at: {public_url}\n")



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



App is running at: NgrokTunnel: "https://3d93-34-142-211-41.ngrok-free.app" -> "http://localhost:5000"



In [ ]:
!ngrok config add-authtoken <Your_NGORK_key>

!pip install flask pyngrok googletrans==4.0.0-rc1 diffusers transformers accelerate huggingface_hub



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
 API_KEY="Your_Hugging_face_key"



In [ ]:
 !pkill -f ngrok

In [ ]:

from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
from googletrans import Translator
from diffusers import StableDiffusionPipeline
from huggingface_hub import login
import torch, io, base64
from PIL import Image
import threading

# Configuration class
class CFG:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seed = 42
    image_gen_steps = 35
    image_gen_model_id = "runwayml/stable-diffusion-v1-5"
    image_gen_size = (600, 600)
    image_gen_guidance_scale = 9
    api_key = API_KEY
    port = 5000

login(CFG.api_key)

# Load the model
pipe = StableDiffusionPipeline.from_pretrained(
    CFG.image_gen_model_id,
    torch_dtype=torch.float16 if CFG.device == "cuda" else torch.float32
).to(CFG.device)

# Flask App
app = Flask(__name__)

# Translation function
def get_translation(text, dest_lang="en"):
    return Translator().translate(text, src=src_lan , dest=dest_lang).text

# Generate images
def generate_images(prompt, model, num_images=3):
    images = []
    for _ in range(num_images):
        image = model(prompt, num_inference_steps=CFG.image_gen_steps,
                      guidance_scale=CFG.image_gen_guidance_scale).images[0]
        image = image.resize(CFG.image_gen_size)
        images.append(image)
    return images

# HTML UI
html_code = """<html><head><title>Image Gen</title></head>
<style>
body{font-family:sans-serif;margin:40px}
#micButton {
  background: #eee; border: none; cursor: pointer;
  padding: 5px 10px; font-size: 16px; margin-left: 10px;
}
</style>
<body><h1>Multilingual Image Generator</h1>

<label>Prompt:
  <input id="textPrompt" style="width:300px">
  <button id="micButton" onclick="startListening()">🎤</button>
</label><br><br>

<label>Language:
  <select id="languageSelect">
    <option value="en">English</option>
    <option value="te">Telugu</option>
    <option value="hi">Hindi</option>
    <option value="es">Spanish</option>
  </select>
</label><br><br>

<button onclick="generateImages()">Generate Images</button>
<div id="output" style="margin-top:20px"></div>

<script>
function generateImages(){
    const prompt = document.getElementById('textPrompt').value;
    const lang = document.getElementById('languageSelect').value;
    document.getElementById('output').innerHTML = "Generating...";
    fetch('/generate-images', {
        method:'POST',
        headers:{'Content-Type':'application/json'},
        body:JSON.stringify({prompt:prompt, language:lang})
    })
    .then(r=>r.json())
    .then(d=>{
        let html = "";
        d.images.forEach((img, i)=>{
            html += `<img src="data:image/png;base64,${img}" width="200"><br><br>`;
        });
        document.getElementById('output').innerHTML = html;
    });
}

// Voice recognition
function startListening() {
    if (!('webkitSpeechRecognition' in window)) {
        alert("Sorry, your browser does not support Speech Recognition.");
        return;
    }
    const recognition = new webkitSpeechRecognition();
    recognition.lang = document.getElementById('languageSelect').value;
    recognition.interimResults = false;
    recognition.maxAlternatives = 1;

    recognition.onresult = function(event) {
        const transcript = event.results[0][0].transcript;
        document.getElementById('textPrompt').value = transcript;
    };

    recognition.onerror = function(event) {
        alert("Error occurred in recognition: " + event.error);
    };

    recognition.start();
}
</script></body></html>"""


@app.route('/')
def index():
    return render_template_string(html_code)

@app.route('/generate-images', methods=['POST'])
def generate_images_endpoint():
    data = request.json
    translated_prompt = get_translation(data['prompt'])
    images = generate_images(translated_prompt, pipe)
    image_data = []
    for img in images:
        buf = io.BytesIO()
        img.save(buf, format='PNG')
        img_base64 = base64.b64encode(buf.getvalue()).decode()
        image_data.append(img_base64)
    return jsonify({'images': image_data})

# Run Flask in a thread
def start_flask():
    app.run(port=CFG.port)

threading.Thread(target=start_flask).start()

# Start Ngrok Tunnel
public_url = ngrok.connect(CFG.port)
print(f"\nApp is running at: {public_url}\n")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.



App is running at: NgrokTunnel: "https://370a-34-91-86-51.ngrok-free.app" -> "http://localhost:5000"

